In [1]:
import bs4
import requests
import pandas

In [2]:
soup = bs4.BeautifulSoup(open('india 1 table 2.html').read(), "html.parser")

soup

<table align="center" border="0" cellpadding="3" cellspacing="0">
<tbody><tr>
<td align="center" class="TableHeading" colspan="5"><b>Information Security Management Systems</b> </td>
</tr>
<tr>
<td align="center" class="TablesubHeading"><strong>Sl.No.</strong></td>
<td align="center" class="TablesubHeading"><b>Accreditation 
              No.</b></td>
<td align="left" class="TablesubHeading"><b>Name of the Certification Bodies</b></td>
<td align="center" class="TablesubHeading">Valid From</td>
<td align="center" class="TablesubHeading"><b>Valid Upto </b></td>
</tr>
<tr>
<td align="center" class="col2" valign="top"><strong>1</strong></td>
<td align="center" class="col1" valign="top"><b>ISMS006</b></td>
<td align="left" class="col1" valign="top"><a href="javascript:;" onclick="MM_openBrWindow('ISO27701/ISO27701.php','MQA','scrollbars=yes,resizable=yes,width=800,height=600')">Bureau Veritas India Pvt. Ltd.</a></td>
<td class="col1" valign="top">08 Jan 2020</td>
<td align="center" class="c

In [3]:
links = soup.find_all('a', href='javascript:;')
links

[<a href="javascript:;" onclick="MM_openBrWindow('ISO27701/ISO27701.php','MQA','scrollbars=yes,resizable=yes,width=800,height=600')">Bureau Veritas India Pvt. Ltd.</a>,
 <a href="javascript:;" onclick="MM_openBrWindow('ISO27701/001-TSSA.php','TSSA','scrollbars=yes,resizable=yes,width=800,height=600')">TUV SUD South Asia Private Limited</a>]

In [8]:
paginationLinks = []
baseURL = 'http://nabcb.qci.org.in/accreditation/'
for link in links:
    tail = link.get('onclick')[17:link.get('onclick').find('php')+3]
    paginationLinks.append(baseURL + tail)

paginationLinks

['http://nabcb.qci.org.in/accreditation/ISO27701/ISO27701.php',
 'http://nabcb.qci.org.in/accreditation/ISO27701/001-TSSA.php']

In [17]:
records = []

for link in paginationLinks:
    soup = bs4.BeautifulSoup(requests.get(link).content)
    title = soup.find('td', align='center', class_='heading').get_text()
    tableRows = soup.find_all('td', class_='justify')[-2].find_all('tr')
    for row in tableRows[1:-1]:
        tableData = row.find_all('td')
        records.append({'Name': title,
                        'IAF Scope': tableData[0].get_text()})

records

[{'Name': 'Bureau Veritas India Pvt. Ltd. ',
  'IAF Scope': '1.       ISO/IEC 27701:2019 \x96 Security techniques - Extension to ISO/IEC 27001 and ISO/IEC 27002 for privacy information management - Requirements and guidelines'},
 {'Name': 'TUV SUD South Asia Private Limited',
  'IAF Scope': '\n 1.  ISO/IEC 27701:2019 - Security techniques - Extension to ISO/IEC 27001 and ISO/IEC 27002 for privacy information management\r\n\t\t\t\t\t- Requirements and guidelines'}]

In [18]:
df = pandas.DataFrame(records)

df.head()

,Name,IAF Scope
0,Bureau Veritas India Pvt. Ltd.,1. ISO/IEC 27701:2019  Security techniq...
1,TUV SUD South Asia Private Limited,\n 1. ISO/IEC 27701:2019 - Security technique...


In [20]:
df.to_csv('NABCB ISO Excel.csv')